# Lab 1: R Basics

This lab will focus on loading data and data cleaning using data in dataframes and/or tibbles.  More basic R, such as data types, arithmetic, etc. is covered in the assigned Intro to R DataCamp course.

- [Loading Data](#load)
- [Checking Data](#check)
    - [Basic df functions](#basic)
    - [Names](#names)
    - [Missing data](#missing)
- [Factors](#factors)
    - [Missing categorical/factor data](#misscat)
    - [Factor levels](#levels)
    - [Collapsing](#coll)
    - [Lumping](#lump)
    - [Numerical variable as factor](#numfct)
- [dplyr verbs](#dplyr)


Before we begin, we first need to load the packages that have the functions we wish to use with the `library()` statement.  Before loading a package for the first time, we need to install it using `install.packages()` function.

In [ ]:
## this is what a comment looks like in R
# install.packages("tidyverse")
## I commented out the line above because I don't need to install tidyverse.  
## But you will need to run this in RStudio the first time 

library(tidyverse)

## Warnings are not a concern.  Errors are a concern.

<a id="load"></a>
## Loading Data

Data can be contained in a number of different types of files.  We will primarily use csv (comma separated values) and .rds files in this course.  

First, let's figure out where we are. Your working director is where your R session is currently in your file structure.  Your R will have a default working directory, which you can change in RStudio through Tools->Global Tools.  Once you load a file, such as an RNotebook (.rmd) file, the working directory will likely be that location.  This becomes important when we want to load data, if the data is not in your current working directory you need to provide a full file path.

In [ ]:
## Get your current working directory
getwd()

## set a different working directory (for this session)
# setwd("~/file/path/here") ## commented because I don't want to run this example

In [ ]:
## You can also run this function to list the files in your current working directory
list.files()

### Loading a .csv file
A .csv file is a "Comma Separated Values" file which is essentially a text file with commas separating the data points.  These will typically also open in Excel, although they are not "Excel files."

I have a .csv file in my current working directory called "small_gss.csv."  I will load it into R.  Note that I use an assignment operator to save the loaded data to a dataframe called "mydata."  If you do not save it you will not be able to access it moving forward.

In [ ]:
#using the tidyverse read_csv() instead of base R read.csv().

mydata <- read_csv("small_gss.csv") ## put the file name, with extension, inside the parentheses

## take a look at a summary of the dataframe to make sure it read in
glimpse(mydata)

That will probably be enough to load most files, but there are optional arguments you may need for some files. View the documentation at https://readr.tidyverse.org/reference/read_delim.html. 

### Loading an Excel file
RStudio/tidyverse allows us to read directly from an Excel file (.xls or .xlsx) without first converting it to .csv.

In [ ]:
library(readxl)  ## this is installed with tidyverse but doesn't get called with library(tidverse)
# so you have to explicitly load it

mydata_xlsx <- read_xlsx("small_gss.xlsx")
glimpse(mydata_xlsx)

### Using .rds format

.rds is an R data format that lets us save an R object (like a dataframe).  We can then later load it back as a dataframe.

In [ ]:
# save mydata to .rds file
# it will save in current working directory
# saveRDS(objname, "filename.rds")
saveRDS(mydata, "mygssdata.rds")

In [ ]:
## load the .rds file as a df called "mydata2"
## dfname <- readRDS("filename.rds")
mydata2 <- readRDS("mygssdata.rds")
glimpse(mydata2)

### Shortcuts
A couple more ways you can make loading data easier.

#### Button in RStudio
<img src="1_basics_images/import.png">

#### Using read_csv (or read_xlsx) with file.choose()
Putting file.choose() instead of a filename in the read_csv() function will prompt your computer to open a file window from which you select the file you wish to open.  Once you click "Open" in the file window the code will finish executing.  Note - the filetype you select in the window must match with the function you use.  You cannot use read_csv to read an .xlsx file even though you can select it in the window.

<img src="1_basics_images/filechoose.png">

<a id="check"></a>
## Checking data
You will notice above that every time I loaded data I immediately ran glimpse() to make sure the data loaded correctly.  There are a few different things we can look at to do a preliminary inspection of our data.

glimpse() is a good high-level glance at your data.  You see the number of observations (rows), the number of variables, a list of all the variable names, variable types, and the first 10-15 observations of each variable, so we can see generally that the data looks the way we expect it to.

In [ ]:
## glimpse
glimpse(mydata)

<a id="basic"></a>
### Basic df functions

There are some other basic inspection functions we can use:

We're going to begin using the pipe operator (%>%)  This operator allows us to perform sequential tasks on a dataframe without nesting function calls.  It comes from the tidyverse package "magrittr."  https://magrittr.tidyverse.org/index.html

Another intro: https://uc-r.github.io/pipe

The pipe operator can be typed in RStudio using the shortcut: CTRL (or command) + SHIFT + m

How it works:

In [ ]:
## THIS WILL NOT RUN

#  %>%  is used to perform sequential tasks.  Here I am the data object, 
#  and all of the actions are going to happen to me in order, the result passing on through the next pipe.

I %>% wakeup()  %>% shower()  %>% dress() %>% makeup() %>% atebreakfast("eggs") %>% commuted("tocampus")

## without the pipe operator, these actions would need to be nested and would be performed from the inside to the outside
## yielding some very unreadable code:

commuted(atebreakfast(makeup(dress(shower(wakeup(I)))), "eggs"),"tocampus")

In [ ]:
## head and tail show us the first X or last X rows of a dataframe
## sometimes helpful when making transformations or edits and want to spot check

## here I'm using the pipe operator.  I call up my dataframe, then I pass it to the function call using %>%
## later you'll see how to string these to do multiple actions at one time.

mydata %>% head() ## 6 rows by default

mydata  %>% tail(10) ## asking for 10 rows

In [ ]:
# dim() tells us the dimensions of the dataset (rows, columns)
dim(mydata)

In [ ]:
# class() tells us the type of an object
class(mydata)
## because we used the tidy version of read_csv our dataframe is also a tibble.

## we can also get the class of a column by using the $ indexing

print("--------------------")  ## printing line to differentiate two different outputs

class(mydata$partyid)

## while mydata$partyid is a vector, class() returns the type of data contained in that vector.

In [ ]:
# summary() gives us descriptive statistics of our variables/columns
summary(mydata)

In [ ]:
## there are often too many variables to run summary on all the variables at once.  
## Instead, we can just specify which columns we want to summarize

summary(mydata$realinc) # use $ indexing to select one column by name

In [ ]:
summary(mydata[15:17]) # use [] indexing to select columns 15 through 17

In [ ]:
summary(select_if(mydata, is.numeric)) ## use select_if and is.numeric to run summary on only the numeric columns

In [ ]:
## if we want to see a frequency table for character columns, we can use table

table(mydata$partyid)

<a id="names"></a>

### Variable names
We can print variable names using the colnames() function.  We can also use this function to rename columns.


In [ ]:
## print variable names
colnames(mydata)

In [ ]:
## save column names as a vector called x
x <- colnames(mydata)
x ## print x

In [ ]:
## change colnames
## make sure you provide a name for every column

## create a vector of new names
newnames <- c("var1", "bob", "why") ## we have 24 variables, but only three names!!! this will be bad.

## change column names to new names
colnames(mydata) <- newnames

## print the column names after changing
colnames(mydata)

Because we didn't provide enough names the remaining variables have missing (NA) names.

Luckily I had saved the previous vector of names as x and can fix it!

In [ ]:
## change column names of mydata to the names in the vector called x
colnames(mydata) <- x 

colnames(mydata) ## confirm the change

<a id = "missing"></a>
### Missing Data
Checking for missing data is the first and most important data cleaning activity.  

For numerical variables, summary() will give us the number of NA values.

In [ ]:
summary(mydata[15:17])

There are 12 NA values in childs and educ, and 17 missing values in age.

what about character (or factor) variables?

In [ ]:
## is.na() returns a vector of T/F indicating which rows have missing values for the variable specified.

## run is.na() for party id variable
is.na(mydata$partyid)[1:30]  ## only printing first 30 values

I only printed the first 30 values because printing 5000+ T/F values is long.  Just having that list is a bit unwieldy. A more helpful thing to do is to get a count of how many values are missing in the column. We can do that by taking the sum() of the vector resulting from is.na() - when we sum boolean values TRUE is 1 and FALSE is 0.

In [ ]:
sum(is.na(mydata$partyid))

There are 65 rows with no value (NA) for party ID.

Now that we know there's missing data, what should we do about it?

The simplest thing to do is remove any rows with any missing data on any of the columns.  We can do this using na.omit()

In [ ]:
dim(mydata) ## get row columns for full df

## create new df that is mydata without any missing
mydata_compcase <- na.omit(mydata)

dim(mydata_compcase) ## print dimensions to show change in number of columns

sum(is.na(mydata_compcase$partyid))  ## number of NA for partyid should be zero

We removed over 1000 rows by removing any row with NA for any of the columns.  Sometimes we may be only interested in conducting an analysis with just 1 or 2 variables and only want to remove the rows that have NA for those variables, while keeping rows that have NA in a variable we're not currently using.  That way we have the most complete data for the variables we're using.

Going back to the full df, mydata, I'm going to remove rows that have NA on age or educ, while keeping rows that have NA on other variables (such as partyid)

In [ ]:
## use  %>% and drop_na with the column names

mydata_dropna  <- mydata  %>% drop_na(age, educ) ## because we're using  %>% to make an action on the df (mydata) 
                                                ## we can just put the variable names like this
dim(mydata) ## dimensions of full
dim(mydata_dropna) ## dimensions of dropna

In [ ]:
summary(mydata_dropna[16:17]) ## summary of age and educ showing no NA

In [ ]:
sum(is.na(mydata_dropna$partyid))  ## still have NA on party ID

Looks like 5 rows that were NA for partyid were also NA for age and/or educ, but the others were kept, as they have data for age and educ.

#### Imputation instead of removing observations

Another way we can deal with missing data is through imputation.  Instead of removing rows with NA for age or education, we can instead replace it with a value.  Some very basic imputations can be done with replacing NA with either the mean or the median of the variable.

In [ ]:
## calculate the mean of age.  We need to use na.rm = TRUE to ignore the NA in the column, otherwise it will return NA
## because any math with an NA is NA such as 4 + 5 + NA + 22 = NA
mean(mydata$age)
mean(mydata$age, na.rm = TRUE)

In [ ]:
## Here I'm selecting the cells of mydata$age that are NA, then assigning those cells the mean of that variable.

#create a copy of mydata
mydataimp <- mydata

summary(mydataimp$age) ## summary with NA

mydataimp$age[is.na(mydataimp$age)] = mean(mydataimp$age, na.rm=TRUE)

summary(mydataimp$age) ## summary after mean imputation

Mean imputation will never change the mean of the variable, but do note that it slightly changed the 75% percentile.

We can also use the median for imputation.  This may be preferred in all the cases where the median is preferred over the mean as a measure of central tendency, such as when the data is skewed.

We'll use median imputation for educ.

In [ ]:
## use the same df created above

summary(mydataimp$educ) ## summary with NA

mydataimp$educ[is.na(mydataimp$educ)] = median(mydataimp$educ, na.rm=TRUE)

summary(mydataimp$educ) ## summary after mean imputation

In this case we didn't remove anything and no other variables were affected, let's again see how many NAs this imputed df has:

In [ ]:
dim(mydata) ## dimensions of full dataset
dim(mydataimp) ## dimensions of the dataset where we did the imputation, the same number of rows
sum(is.na(mydataimp$partyid))  ## still have all 65 NA on party ID

In order to start looking at missing on character variables, we need to first learn about factors.

<a id = "factors"></a>

## Factors

Factors are a way to convert character data to a categorical variable with a finite number of values.  Under the hood they are saved as integers, which correspond to a factor label.

Basic intro to factors - https://www.stat.berkeley.edu/~s133/factors.html

The first step is to convert the variable to a factor.

In [ ]:
## create new column partyid_cat and save the factorized version of partyid
mydata$partyid_cat <- factor(mydata$partyid)
class(mydata$partyid_cat)

In [ ]:
##summary of factor variable
summary(mydata$partyid_cat)

As opposed to the non-informational summary we got above, that just told us that partyid was a character variable, we now get a list of the factor levels and a count of each.

In [ ]:
## show the structure of the new factor variable
str(mydata$partyid_cat)

You can see in the above output that this variable is a factor w/ 8 levels (NA doesn't count as a level), it shows the first label, then in the listing of the first few observations we see the underlying numerical representation (without the labels). 

If we run head() we'll see the labels as the cell values instead.

In [ ]:
mydata  %>% 
    select(id_, partyid, partyid_cat)  %>% ##select only these columns to print
    head(10) ## print first 10 rows

<a id = "misscat"></a>
### Missing Categorical/Factor Data

So we know we still have 65 NAs in partyid_cat, and it's not considered a factor level.  We can do one of two things:

1. remove rows where partyid_cat is NA (see instructions above using drop.na())

**OR**

2. make "missing" it's own category/factor level

In [ ]:
## use fct_explicit_na() to add a 9th level to partyid_cat, "Missing"
## fct_explicit_na(variable, na_level = "LABEL for your NA category/level")

mydata$partyid_cat <- fct_explicit_na(mydata$partyid_cat, na_level = "Missing")

summary(mydata$partyid_cat)

Having an explicit missing category lets us mitigate bias in our analysis by recognizing that some respondents may not want to answer a question, and that unwillingness to answer should be addressed in our analysis.

<a id="levels"></a>
### Factor Levels

There are some other useful functions for performing data cleaning and recoding on factor variables.  forcats is the tidyverse package that contains these functions - https://www.rdocumentation.org/packages/forcats/versions/0.4.0

First we'll look at the labels of the factor levels

In [ ]:
## print the levels of partyid_cat
levels(mydata$partyid_cat)
y <- levels(mydata$partyid_cat)

In [ ]:
# we can rename them the same way we renamed columns using colnames, this time with levels()

#create a vector of new factor level labels, 9 in total.  They need to be in the same order as the labels above

pid_labels <- c("demind", "repind", "ind", "nsdem", "nsrep", "other", "strdem", "strrep", "no answer")

levels(mydata$partyid_cat) <- pid_labels
levels(mydata$partyid_cat)

In [ ]:
## use the saved labels, y, to put the old labels back
levels(mydata$partyid_cat) <- y
summary(mydata$partyid_cat)

In [ ]:
## change just some labels using fct_recode()

## saving the result to a different variable - we could use the same name to overwrite

mydata$partyid_cat2 <- fct_recode(mydata$partyid_cat, ind = "Independent")
summary(mydata$partyid_cat2)

In [ ]:
## we can also use fct_recode to recode several levels to the same name, essentially combining those levels.

mydata$partyid_cat2 <- fct_recode(mydata$partyid_cat, ind = "Independent", ind = "Ind,near dem", ind = "Ind,near rep")
summary(mydata$partyid_cat2)

In [ ]:
## here's a way to collapse multiple categories using fct_collapse

fct_count(mydata$partyid_cat)

partyid2 <- fct_collapse(mydata$partyid_cat,
  missing = "Missing",
  other = "Other party",
  rep = c("Strong republican", "Not str republican"),
  ind = c("Ind,near rep", "Independent", "Ind,near dem"),
  dem = c("Not str democrat", "Strong democrat")
)

fct_count(partyid2)

Notice that in addition to collapsing the factors as we specified, it also reordered the factor levels.  We can change this order if we want to using fct_relevel.

In [ ]:
summary(mydata$partyid_cat)

## move Independent to the first position
p2 <- fct_relevel(mydata$partyid_cat, "Independent")

summary(p2)

In [ ]:
## move Other party to the last position
p2 <- fct_relevel(p2, "Other party", after = Inf) ## Inf is used to say after the last value.

summary(p2)

In [ ]:
## Put Missing after the first three "ind" levels
p2 <- fct_relevel(p2, "Missing", after = 3) ## Inf is used to say after the last value.

summary(p2)

In [ ]:
## sort alphabetically by label
p3 <- fct_relevel(p2, sort)
summary(p3)

In [ ]:
## reverse current level order
p3 <- fct_relevel(p3, rev)
summary(p3)

In [ ]:
## put factors in order by frequency (largest first)
p3 <- fct_infreq(p3)
summary(p3)

In [ ]:
## I can also specify the exact order I want, here creating a political spectrum

p4 <- fct_relevel(p3, "Strong democrat", "Not str democrat", 
                  "Ind,near dem", "Independent", "Ind,near rep", "Not str republican", "Strong republican")

## because I wanted to leave other and missing at the end like they were I didn't have to include them in the relevel
summary(p4)

<a id="coll"></a>
### Collapsing
We saw above in the renaming factors section how to use factor collapse to specify new categories and collapse the existing categories into those new categories based on exactly how we want it to happen.  Reminder:


In [ ]:
## here's a way to collapse multiple categories using fct_collapse

fct_count(mydata$partyid_cat)

partyid2 <- fct_collapse(mydata$partyid_cat,
  missing = "Missing",
  other = "Other party",
  rep = c("Strong republican", "Not str republican"),
  ind = c("Ind,near rep", "Independent", "Ind,near dem"),
  dem = c("Not str democrat", "Strong democrat")
)

fct_count(partyid2)

With fct_collapse we have full control of which of the smaller groups goes into which larger groups.

<a id = "lump"></a>
### Factor Lumping

Another thing we can do is fct_lump() when we have a bunch of small groups and want to just lump them together as an "other" category.  Make sure this makes sense for your data - you don't want to lump together a bunch of things that don't logically go together in a way that might affect your analysis.  We wouldn't want to, for example, have both strong democrat and strong republican in one other category.

For this example I'm going to use the religion variable.

In [ ]:
#here instead of making a new factor variable, I'm going to overwrite the same variable.
# If I make a mistake doing this there is no "undo" - 
## you would have to load the data again from the top and then rerun all of the previous data cleaning steps.
mydata$relig <- factor(mydata$relig)
summary(mydata$relig)

I notice 2 things here:

1. There are NAs, which we know how to deal with.

2. There are many groups with very small counts (Native american with 5, for example)

In this case I'm going to do the lumping before I deal with the NAs. This way the NAs will not be lumped into the "other" category.

In [ ]:
## print summary in order by frequency

## because I'm not saving this, It's not changing my data, 
## just changing the order the levels are printed when I call summary
summary(fct_infreq(mydata$relig))

In [ ]:
## lump with default

mydata$lump1 <- fct_lump(mydata$relig) ## instead of overwriting relig, saving as new column called lump1

## print summary of result after lumping
summary(fct_infreq(mydata$lump1))

I can now deal with the NAs using fct_explicit_na()


In [ ]:
mydata$lump1 <- fct_explicit_na(mydata$lump1)
summary(fct_infreq(mydata$lump1))

If you want to have more control over the lumping, you can use an argument (n) that preserves the most common n values.
We can also explicitly label the other category with our own label using other_level

In [ ]:
## lump with arguments, keep 4 most common religions

mydata$lump2 <- fct_lump(mydata$relig, n = 4, other_level = "Other religions") 

## print summary of result after lumping
summary(fct_infreq(mydata$lump2))

We kept the 4 most common religions in the data - Protestant, None, Catholic, and Jewish.  We gave our other category a custom name.  We didn't deal with the NAs, so they remain.  Jewish is displayed below Other in the list sorted by frequency because even though it was the 4th most common, it's smaller than the rest of the groups lumped together.

#### Dropping unused Factor Levels
Sometimes when you're preparing to conduct your analysis you might end up subsetting your data to only analyze some of the observations.  For example, when we do a t-test we compare the means between two groups.  If I wanted to use mydata to compare the mean of income by religion, I might decide to select the two largest groups to compare, "Protestant" and "None."

I'm going to first need to filter my data to obtain the subset (more about this below).

In [ ]:
#create a subset of mydata that contains only the people who are Protestant or "None" on relig
sub_mydata <- mydata  %>% filter(relig == "Protestant" | relig == "None") ## | is the OR operator

##look at difference in the two dfs, the full and the subset

summary(mydata$relig)
summary(sub_mydata$relig)


You can see that the other observations were removed, but the factor levels remain.  These unused factor levels will show up in your frequency tables and can affect your ability to run your analyses correctly.  Before proceeding, we can remove the unused factor levels.

In [ ]:
## use droplevels to remove unused factor levels

sub_mydata$relig <- droplevels(sub_mydata$relig)
summary(sub_mydata$relig)

<a id = "numfct"></a>
### Converting numerical/continuous variable to a factor

For some analyses we will need to use categorical/ordinal variables, but our data is continous/numerical.  We can split our numerical variable into ranges in order to use it as a categorical variable.  

For example, a chi-square analyses (which we will learn soon) measures whether two categorical variables are related.  If we wanted to see if there was an association between income and partyid using that analysis, we would first need to convert income to a categorical variable.

In [ ]:
## review summary of income

summary(mydata$realinc)

In [ ]:
### remove observations (rows) where realinc is missing and save to new df (so I'm not overwriting mydata)
mydata2 <- mydata %>% drop_na(realinc)
summary(mydata2$realinc)

We can see that the lowest income is 227 and the highest is 131,677.  We need to decide which levels we want to create to make a categorical/factor variable.  The cut() function is used to create a categorical variable from a numerical variable.  By default cut() will create ranges of equal size based on the number of breaks you want.  We'll create a variable with ten levels of income to start.

In [ ]:
mydata2$inc_cut10 <- cut(mydata2$realinc, br = 10) ## br = 10 says we want 10 levels
table(mydata2$inc_cut10)

In our table we get 10 groups of equal width.  The first group (95.6,1.34e+04] reflects the range of incomes greater than 95.6 dollars and less than or equal to 13400 (1.34e+04 is scientific notation for 1.34 x 10^4 which is 13,400).  1427 people in our data have incomes between those values.  

Some issues with this method:

- By cutting the full range of values into 10 groups of equal range, we end up with groups that are harder to understand. `$0 to $10,000` makes a lot more sense than `$95.60 to $13,400`.
- Some of our groups have zero observations within them because there are no incomes between 79,100 and 119,000.  Instead we would probably decide to use something like `$80,000 and higher` to capture the top 335 observations without having empty groups.

So in order to avoid these issues, I am going to specify exactly where I want to make the cuts.

First, let me lay out the categories I want to end up with:
<br>`$0 to 10k`
<br>`$10k to 20k`
<br>`$20k to 30k`
<br>`$30k to 40k`
<br>`$40k to 50k`
<br>`$50k to 60k`
<br>`$60k to 70k`
<br>`$70k to 80k`
<br>`$80k and above`

So I want to have 9 categories in total.  The cutoffs between the categories are:

10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000

for a total of 8 cutoffs between levels.

We also need to add a value to define the start of the bottom level and the end of the top level.  Typically we can use -Inf and Inf (negative infinity and positive infinity) to denote those.  So in order to have 9 levels we need to provide 10 cutoff values

In [ ]:
# create a vector that defines our 10 cutoffs
cutoffs <- c(-Inf, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, Inf)

## use the cut function with these predefined cutoffs
mydata2$inc_cut <- cut(mydata2$realinc, br = cutoffs) ## use our saved vector of cutoffs for the breaks
summary(mydata2$inc_cut)

This looks a lot better, but let's relabel those levels (they aren't pretty this way). 

In [ ]:
## define a vector of "pretty" labels
inclabels <- c("$0 to 10k", "$10k to 20k", "$20k to 30k", "$30k to 40k", "$40k to 50k", "$50k to 60k", "$60k to 70k",
               "$70k to 80k", "$80k and above")

## use levels() to relabel the factor
levels(mydata2$inc_cut) <- inclabels

summary(mydata2$inc_cut)

<a id = "dplyr"></a>

## dplyr verbs

dplyr is considered to be the grammar of data manipulation.  It is a set of 5 verbs that allow us to accomplish a variety of data manipulation tasks

<img src="1_basics_images/dplyr.jpg" width = "500">


The verbs are:
- arrange()
- select()
- filter()
- mutate()
- summarize() or summarise()

For these examples I'm going to begin with a very basic dataframe (tibble).

In [ ]:
## Creating a dataframe of 5 classmembers with their names, gender, 
## number of hamsters they have and the number of hamster cages they have.

hamsters <-
    tibble(
        name = c('Megan', 'Amy', 'Jen', 'Karl', 'Jeremy'),
        gender = c('female', 'female', 'female', 'male', 'male'),
        ham_num = c(5, 7, 6, 2, 1), # number of hamsters
        hamster_cages = c(2, 1, 3, 3, 4) # number of cages
        )

hamsters

### Arrange
Arrange sorts your dataset by the variable you specify.

In [ ]:
## note I'm running this and it prints the result, but it's not being saved and therefore not changing anything about hamsters
hamsters %>% arrange(ham_num)

## if i did:
## hamsters <- hamsters %>% arrange(ham_num) 
## that would overwrite hamsters with the arranged version

This sorted by ham_num from lowest to highest.  If I want to get from highest to lowest I need to use desc() for descending.

In [ ]:
hamsters %>% arrange(desc(ham_num))

In [ ]:
## if I sort by a character variable I get alphabetized results
hamsters %>% arrange(name)

In [ ]:
## I can sort by two variables, the second used for breaking ties
hamsters %>% arrange(gender, name)

### Select
Select allows us to select specific columns from the overall dataframe.  It may seem a bit boring right now, but later it will prove useful.

In [ ]:
## select 2 columns by variable name
hamsters %>% select(name, ham_num)

In [ ]:
## a negative sign with a variable name will select everything BUT that column
hamsters %>% select(-name)

In [ ]:
## select can be used to reorder columns
hamsters %>% select(ham_num, hamster_cages, gender, name)

We can now build longer pipes and string together multiple actions.  I'm going to use select, then arrange.

In [ ]:
hamsters %>% select(ham_num, hamster_cages) %>% arrange(ham_num)

### Filter
Filter is used in conjuction with a logical expression to retrieve rows from the df which that specification.

First I'll use filter to just return those students who are female.

In [ ]:
hamsters %>% filter(gender == "female")

Two things to pay attention to:
1. What you put in the filter function (the logical statement) should reflect what you want to KEEP (the females) and not what you want to remove.

2. The difference between logical and mathematical operators.  `==` is "equal to", not `=`

There's a list of logical operators here: https://www.statmethods.net/management/operators.html

Let's go back to a variable with more values for a minute, partyid.  Say we want to subset our data to keep everyone who didn't say "Other Party."

In [ ]:
## reminder of the values
mydata %>% count(partyid_cat)

In order to make a logical for everything we want to keep, we may think we need to list them all out with "or", like:

partyid_cat == "Ind,near dem" | partyid_cat == "Ind, near rep" | etc. etc. etc.

and just leave out the one we don't want.  However, in this case it's just easier to say "not" other party (the not operator is !)

In [ ]:
mydata %>% filter(partyid_cat != "Other party")  %>% count(partyid_cat)

So we did two actions, first did a filter, then created the frequency table using count().  If I wanted to save the subsetted dataset created by the filter I would not want to include count() at the end of the pipe, as then it would save just the frequency table.

In [ ]:
## DON'T DO THIS

badsubset <- mydata %>% filter(partyid_cat != "Other party")  %>% count(partyid_cat)
badsubset

## you've saved just the frequency table and not the underlying observations

In [ ]:
## DO THIS

correctsubset <- mydata %>% filter(partyid_cat != "Other party")
head(correctsubset)


In [ ]:
## now I can do the frequency on the saved subset
correctsubset %>% count(partyid_cat)

### Mutate
Mutate is a very powerful verb.  It lets us create new variables in our dataframe.  We're going to go back to the hamsters example first

In [ ]:
hamsters %>% mutate(hamsters_per_cage = ham_num / hamster_cages)



REMEMBER - we didn't save this so this has not made any changes to hamsters.  In order to keep the new variable we would have to add the assignment operator:




In [ ]:
# save the new df with the added variable as hamsters2
hamsters2 <- hamsters %>% mutate(hamsters_per_cage = ham_num / hamster_cages)

## to show nothing happened to hamsters when we ran the code above
print("HAMSTERS")
hamsters

## but hamsters2 includes the new variable, because we saved the output of the pipe to an object called hamsters2
print("HAMSTERS2")
hamsters2

In [ ]:
## FANCY magrittr assignment pipe
# IF you want to save the changes you make to your df in your pipe
# AND you're ok overwriting your current df
# you can use this special pipe operator

# the basic pipe operator is called by library(tidyverse), but the others need you to explicitly load magrittr
library(magrittr)

hamsters2 %<>% select(-gender) ## select every column from hamsters2 except gender
#  %<>% is the operator that pipes forward AND assigns back, overwriting that same df

hamsters2


We can add completely new variables with mutate.  I want to add the number of cats each student has to the df.  Note, the vector needs to be in the order of the observations/rows.  Megan's number of cats first, and Jeremy's last.

In [ ]:
## create the vector for the new column
catsvals = c(4, 5, 2, 1, 3) ## number of cats for students in order of observations in df

## use assignment/pipe to add variable called cats with the values in the vector and save it, overwritting hamsters
hamsters %<>% mutate(cats = catsvals)

hamsters

In [ ]:
## I can also add a new variable where every observation gets the same value
hamsters %>% mutate(walruses = 0)
# note - not saving this time

#### Transmute
Transmute is a version of mutate() that only returns the new columns created at that time.

In [ ]:
hamsters %>%
    transmute(hamsters_per_cage = ham_num / hamster_cages,
                five_or_more_hamsters = ham_num >= 5)

### Summarize (or Summarise)
The creator of RStudio and the tidyverse is from New Zealand, so "British English" spellings are typically aliases for the American English spelled functions.

Summarize() lets us summarize our data.  The output from summarize is a df with just the summary information.

In [ ]:
#single mean
hamsters %>% summarize(hamsters_mean = mean(ham_num))

By saying hamsters_mean = mean(ham_num), we're naming our summary value hamsters_mean.  We can name it whatever we want.

In [ ]:
#single mean
hamsters %>% summarize(average_ham = mean(ham_num))

In [ ]:
# no label
hamsters %>% summarize(mean(ham_num))

In [ ]:
## summarizing both means and medians of 2 different variables
hamsters %>%
    summarise(hamsters_mean = mean(ham_num),
                hamsters_median = median(ham_num),
                hcmean = mean(hamster_cages),
                hcmedian = median(hamster_cages))

Remember, if you have any missing data ("NA") you will need to use na.rm = TRUE when calling the mean or median functions

In [ ]:
mydata %>% summarize(meanage = mean(age)) ### WRONG - AGE has NAs

mydata %>% summarize(meanage = mean(age, na.rm = TRUE))

Group_by() makes summarize() even more powerful - we can get summaries by group

In [ ]:
hamsters %>%
        group_by(gender) %>%
        summarize(mean_hamsters = mean(ham_num),
                    max_hamsters = max(ham_num),
                    num_people = n()) ## n() used in a summarize after a group_by will give the number of obs in that group

In our summary we get two rows - one row for each group (here male and female), and the 3 summary statistics we requested, the mean of hamsters, the max of hamsters (largest number of hamsters in that group) and the number of observations in the group, using n().

We can use any combination of pipes and verbs to manipulate our data:

In [ ]:
new_df <- hamsters %>% ## saving the result to new_df
          arrange(ham_num) %>% ## sort by ham_num low to high
          select(-name) %>% ## select all columns BUT name
          mutate(walruses = 0) %>% ## add a new variable/column called walruses where every observations gets the value 0
          group_by(gender) %>% ## group the observations by gender - this is for the calculation of mean below
          mutate(hamsters_center = ham_num - mean(ham_num)) ## create a new variable, hamsters_center, which is 
            # the difference between the observation's number of hamsters and the mean (within the group) number of hamsters
new_df ## printing new_df

Notice that the group_by(gender) didn't limit us to two rows outputted.  This is because we didn't use summarize.  Instead, we grouped by gender, then that grouping was taken into account when calculating the mean in the following mutate.

These tools are powerful, but take practice to learn how to build up and use together.  I always suggest building your pipes step by step and reviewing results, not overwriting your df in the process.  Review your output at each step and make sure it's doing what you want it to do.  THEN, once you know it's right, you can save the final result.